Importing libraries

In [1]:
!pip install transformers accelerate datasets peft bitsandbytes pandas
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 14.3 MB/s eta 0:00:00


Extract the files to access the checkpoint

In [2]:
import zipfile

zip_path = '/content/qwen3_finetuned.zip'

extract_path = '/content/qwen3-4b-med-multiclass/'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
  zip_ref.extractall(extract_path)

Importing data into dataframes

In [3]:
import pandas as pd

file_path = "train-test-data.xlsx"

train_df = pd.read_excel(file_path, sheet_name="train")
test_df  = pd.read_excel(file_path, sheet_name="test")

print("Train dataset sample:")
display(train_df.head())

print("Test dataset sample:")
display(test_df.head())

Train dataset sample:


,case_id,input_finding,output_disease
0,1,The liver surface is regular with no apparent ...,Liver lesion
1,2,"The liver and spleen are in normal position, s...","Kidney stone, Pancreatitis"
2,3,"The liver surface is smooth, and the size and ...","Liver cyst, Renal cyst, Splenic cyst"
3,4,"Abnormal liver morphology, with arc-shaped low...","Liver lesion, Liver cyst, Gallstone, Ascites, ..."
4,5,"Two round low-density lesions, approximately 5...","Liver lesion, Splenomegaly, Renal mass, Colore..."


Test dataset sample:


,case_id,input_finding,output_disease
0,1,The liver is normal in size and shape with hom...,"Renal cyst, Adrenal hyperplasia, Adrenal calci..."
1,2,"The liver is normal in position, size, and sha...","Liver lesion, Atherosclerosis"
2,3,"The liver surface is smooth, with coordinated ...","Liver cyst, Pancreas calcifications, Accessory..."
3,4,The liver surface is smooth with coordinated s...,"Cholecystitis, Renal cyst, Ascites"
4,5,"The surface of the liver is smooth, and the si...","Liver calcifications, Gallstone"


Inference Code to load checkpoints the way they were fine tuned using LoRA. Make sure to load them as is in order to have same layer mapping and avoiding issues.

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

model_name = "Qwen/Qwen3-4B"
lora_path = "/content/qwen3-4b-med-multiclass/qwen3-4b-med-multiclass-finetuned/"

bnb_config = BitsAndBytesConfig(load_in_4bit=True)

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype="auto",
    trust_remote_code=False,   # must match training
)

model = PeftModel.from_pretrained(
    model,
    lora_path,
    is_trainable=False
)

model.eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 2560)
        (layers): ModuleList(
          (0-35): 36 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2560, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linea

In [5]:
from sklearn.preprocessing import MultiLabelBinarizer
df = pd.read_excel("train-test-data.xlsx")
label_lists = [list(dict.fromkeys(lbl.split(", "))) for lbl in df['output_disease']]

mlb = MultiLabelBinarizer()
labels_encoded = mlb.fit_transform(label_lists)

classes = mlb.classes_
print("All disease classes:", classes)
print("Multi-label encoded shape:", labels_encoded.shape)

All disease classes: ['Abdominal abscess' 'Abdominal aortic aneurysm' 'Abdominal lipiditis'
 'Abdominal mass' 'Absent kidney' 'Accessory spleen'
 'Adrenal calcification' 'Adrenal hyperplasia' 'Adrenal mass'
 'Adrenal metastasis' 'Adrenal nodule' 'Annular pancreas' 'Appendicitis'
 'Arteriosclerosis' 'Ascites' 'Atherosclerosis' 'Basal cell carcinoma'
 'Bile duct cancer' 'Bile duct carcinoma' 'Bile duct dilatation'
 'Bile duct stone' 'Cecal epiploic appendagitis' 'Cholecystitis'
 'Cirrhosis' 'Colitis' 'Colorectal cancer' 'Diverticulitis' 'Enteritis'
 'Enteritis possible' 'Epiploic appendagitis' 'Fatty liver'
 'Gallbladder mass' 'Gallstone' 'Gastric cancer' 'Gastric lymphoma'
 'Gastrointestinal bleed' 'Gastrointestinal perforation' 'Hepatitis'
 'Hernia' 'Hydronephrosis' 'Inflammatory bowel disease'
 'Intestinal obstruction' 'Kidney infarction' 'Kidney stone'
 'Liver abscess' 'Liver calcification' 'Liver cyst' 'Liver lesion'
 'Lymphadenopathy' 'Neuroblastoma' 'No acute abnormality'
 'Omenta

Inference on the saved checkpoints

In [ ]:
import torch
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

labels_str = ", ".join(classes)
batch_size = 4

def build_prompt(report):
    if len(report) > max_input_chars:
        report = report[:max_input_chars] + " [TRUNCATED]"

    return f"""
You are a radiology assistant. Extract all diseases mentioned in the report.
Use only the known disease classes listed below. Do not invent any new diseases.
Return your answer as a comma-separated list.

Known disease classes: {labels_str}

Report: "{report}"
Output:
"""

# --------------------------------------------------------
# BATCH INFERENCE
# --------------------------------------------------------
preds = []
output_file = "test_predictions.csv"

for i in tqdm(range(0, len(test_df), batch_size), desc="Processing"):
    batch = test_df.iloc[i:i+batch_size]

    prompts = [build_prompt(txt) for txt in batch["input_finding"]]

    # Tokenize together (fast)
    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True
    ).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=20,
            do_sample=False
        )

    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # Strip prompt from model output
    for p, full_output in zip(prompts, decoded):
        prediction = full_output.replace(p, "").strip()
        preds.append(prediction)

    # Save progress
    if i % (batch_size * 5) == 0:
        pd.DataFrame({"predicted": preds}).to_csv(output_file, index=False)

# Final save
test_df["predicted_disease"] = preds
test_df.to_csv(output_file, index=False)


Processing:  22%|██▏       | 21/97 [11:02:07<39:57:49, 1893.01s/it]A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
